In [1]:
import numpy as np
from sklearn.cluster import KMeans
import torch
import matplotlib.pyplot as plt
import pandas as pd
import torch
from sklearn.manifold import TSNE
import numpy as np
import matplotlib
import japanize_matplotlib
import glob
import sys
import optuna
sys.path.append("C:/Users/yuuta/Documents/fashion")
from utils.util import open_json
from utils.util import filter_basic_items

c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = glob.glob(
    # "C:/Users/yuuta/Documents/fashion/data/test/**/*.json"
    "C:/Users/yuuta/Documents/fashion/data/train/**/*.json"
)

In [3]:
id_to_vector = open_json('C:/Users/yuuta/Documents/fashion/model_learning/versatility/data/id_to_vector_train.json')

In [22]:
labels = []
vectors = []

In [23]:
for fp in path:
    json_dict = open_json(fp)
    items = filter_basic_items(json_dict["items"])
    for item in items:
        labels.append(item["category x color"])
        itemId = str(item['itemId'])
        v = torch.tensor(id_to_vector[itemId])
        vectors.append(v)

KeyboardInterrupt: 

In [ ]:
labels_list = list(set(labels))
labels = [labels_list.index(l) for l in labels]

In [ ]:
tensor = torch.stack(vectors).numpy()

In [ ]:
p = torch.ones(len(tensor))
index = p.multinomial(num_samples=10000, replacement=False)
sampling_tensor = tensor[index]
sampling_label = np.array(labels)
sampling_label = sampling_label[index]

In [17]:
def classification_scorer(X, Y):
    s = 0
    for i, (x1, y1) in enumerate(zip(X, Y)):
        for j, (x2, y2) in enumerate(zip(X, Y)):
            if i > j and y1 != y2:
                dist = np.linalg.norm(np.array(x1) - np.array(x2))
                s += dist
    return s 

In [19]:
class SupervisedTSNE:
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        self.scorer = classification_scorer
        self.best_score = 0
        self.best_model = None

    def __call__(self, trial):
        perplexity = trial.suggest_float("perplexity", 5, 50)
        early_exaggeration = trial.suggest_float("early_exaggeration", 6, 48)
        init = trial.suggest_categorical("init", ["random", "pca"])

        mapper = TSNE(
            n_components=2,
            random_state = 0,
            n_iter = 1000,
            perplexity=perplexity,
            early_exaggeration=early_exaggeration,
            init=init
        )
        embedding = mapper.fit_transform(self.X)
        score = self.scorer(embedding, self.Y)

        if self.best_score > score:
            self.best_score = score
            self.best_model = mapper
            
            print(self.best_model)
            title='trial={0}, score={1:.3e}'.format(trial.number, score)
            plt.title(title)
            plt.scatter(embedding[:, 0], embedding[:, 1], c=self.Y, alpha=0.5)
            plt.colorbar()
            plt.show()      

        return score

In [ ]:
objective = SupervisedTSNE(sampling_tensor, sampling_label)
study = optuna.create_study(direction="maximize")
study.optimize(objective, timeout=7200 * 2)

print('params:', study.best_params)

[I 2024-01-13 18:42:42,490] A new study created in memory with name: no-name-c6188a6a-1284-4c86-b2ff-c5c5b25f972d
[W 2024-01-13 18:55:22,023] Trial 0 failed with parameters: {'perplexity': 15.507873105214221, 'early_exaggeration': 40.49771604921961, 'init': 'pca'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\yuuta\AppData\Local\Temp\ipykernel_20776\3420999893.py", line 23, in __call__
    score = self.scorer(embedding, self.Y)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\yuuta\AppData\Local\Temp\ipykernel_20776\2990370223.py", line 6, in classification_scorer
    dist = np.linalg.norm(np.array(x1) - np.array(x2))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\n

KeyboardInterrupt: 

In [ ]:
fig = plt.figure(figsize = (50, 50))
colors = list(matplotlib.colors.CSS4_COLORS.values())
for i in range(100):
    category_mask = sampling_label == np.array(i)
    tmp_df = X[category_mask]
    plt.scatter(tmp_df['col1'],
                tmp_df['col2'],
                color = colors[i])